In [5]:
import sys
print(sys.path)


['e:\\Project\\dsr32\\12NLP_Transfer_Learning', 'c:\\Users\\DAnand\\anaconda3\\envs\\tf-gpu\\python39.zip', 'c:\\Users\\DAnand\\anaconda3\\envs\\tf-gpu\\DLLs', 'c:\\Users\\DAnand\\anaconda3\\envs\\tf-gpu\\lib', 'c:\\Users\\DAnand\\anaconda3\\envs\\tf-gpu', '', 'C:\\Users\\DAnand\\AppData\\Roaming\\Python\\Python39\\site-packages', 'c:\\Users\\DAnand\\anaconda3\\envs\\tf-gpu\\lib\\site-packages', 'c:\\Users\\DAnand\\anaconda3\\envs\\tf-gpu\\lib\\site-packages\\win32', 'c:\\Users\\DAnand\\anaconda3\\envs\\tf-gpu\\lib\\site-packages\\win32\\lib', 'c:\\Users\\DAnand\\anaconda3\\envs\\tf-gpu\\lib\\site-packages\\Pythonwin']


In [3]:
import tensorflow as tf
from tensorflow.keras import models, layers
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np

ModuleNotFoundError: No module named 'tensorflow.keras'

In [ ]:
datasets, info = tfds.load(
    "imdb_reviews/plain_text",
    split=["train", "test[:50%]", "test[50%:]"],
    as_supervised=True,
    with_info=True
)
dataset_train_original = datasets[0]
dataset_validate_original = datasets[1]
dataset_test_original = datasets[2]
info

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [ ]:
dataset_train = dataset_train_original.cache()
dataset_train = dataset_train.shuffle(25_000)
dataset_train = dataset_train.batch(128)

dataset_validate = dataset_validate_original.cache()
dataset_validate = dataset_validate.batch(128)

dataset_test = dataset_test_original.batch(128)

In [ ]:
def render_history(history):
    plt.title("Our losses")
    plt.plot(history.history["loss"], label="loss")
    plt.plot(history.history["val_loss"], label="val_loss")
    plt.legend()
    plt.show()
    plt.close()

    plt.title("Our accuracies")
    plt.plot(history.history["accuracy"], label="accuracy")
    plt.plot(history.history["val_accuracy"], label="val_accuracy")
    plt.legend()
    plt.show()
    plt.close()

In [ ]:
vocabulary_size = 10_000
sequence_length = 128

encoder = layers.TextVectorization(
    max_tokens=vocabulary_size,
    output_sequence_length=sequence_length,
    standardize="lower_and_strip_punctuation",
    split="whitespace",
    output_mode="int"
)
encoder.adapt(dataset_train_original.map(lambda text, label: text).batch(512))

In [ ]:
model = models.Sequential()
model.add(encoder)
model.add(layers.Embedding(input_dim=vocabulary_size, output_dim=32))
model.add(layers.LSTM(64))
model.add(layers.Dense(1, activation="sigmoid"))
model.summary()

In [ ]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    dataset_train,
    epochs=30,
    validation_data=dataset_validate
)

In [ ]:
# history = model.fit(
#     dataset_train,
#     epochs=10,
#     validation_data=dataset_validate
# )

In [ ]:

render_history(history)